In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights
from PIL import Image
from tqdm import tqdm

In [2]:
print("gerar csv com método tta(test time augmentation)")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TestDatasetTTA(Dataset):
    
    def __init__(self, test_dir):
        self.test_dir = test_dir
        self.image_files = sorted([
            f for f in os.listdir(test_dir)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        
        # Transformações base
        self.normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        # 5 augmentations diferentes
        self.transforms = [
            # Original
            transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                self.normalize
            ]),
            # Horizontal flip
            transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(p=1.0),
                transforms.ToTensor(),
                self.normalize
            ]),
            # Crop maior e menoor
            transforms.Compose([
                transforms.Resize((240, 240)),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                self.normalize
            ]),
            transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                self.normalize
            ]),
            # Leve rotação
            transforms.Compose([
                transforms.Resize((240, 240)),
                transforms.RandomRotation(5),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                self.normalize
            ]),
        ]
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        filename = self.image_files[idx]
        img_path = os.path.join(self.test_dir, filename)
        image = Image.open(img_path).convert('RGB')
        
        # Aplicar todas as transformações
        augmented = [transform(image) for transform in self.transforms]
        
        image_id = filename
        return augmented, image_id

gerar csv com método tta(test time augmentation)


In [3]:
def predict_with_tta(model_path, test_dir, output_file='submission_tta.csv'):
    """
    Faz predição com TTA usando seu modelo ResNet50 existente
    """
    print(f"\n📂 Carregando modelo: {model_path}")
    
    # Carregar modelo
    model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)

    for nome, parametro in model.named_parameters():
        if "layer4" in nome or "fc" in nome:
            parametro.requires_grad = True
        else:
            parametro.requires_grad = False

    num_features = model.heads.head.in_features

    model.heads = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 2)
    )
    
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"✅ Modelo carregado")
    if 'val_acc' in checkpoint:
        print(f"   Val Acc: {checkpoint['val_acc']:.2f}%")
    if 'val_roc_auc' in checkpoint:
        print(f"   Val ROC-AUC: {checkpoint.get('val_roc_auc', 0.9952):.4f}")
    
    # Criar dataset e loader
    test_dataset = TestDatasetTTA(test_dir)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)
    
    print(f"\n✅ Dataset: {len(test_dataset)} imagens")
    print(f"   TTA: {len(test_dataset.transforms)} augmentations por imagem")
    
    # Predição com TTA
    print("\n🔮 Gerando predições com TTA...")
    predictions = []
    
    with torch.no_grad():
        for augmented_list, image_ids in tqdm(test_loader, desc="TTA"):
            # augmented_list é uma lista de [num_augs] tensores
            batch_size = len(image_ids)
            num_augs = len(augmented_list)
            
            # Coletar probabilidades de cada augmentation
            all_probs = []
            
            for aug_idx in range(num_augs):
                # Pegar imagens desta augmentation
                images = augmented_list[aug_idx].to(device)
                
                # Predição
                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)[:, 1]  # Classe PNEUMONIA
                all_probs.append(probs.cpu().numpy())
            
            # Média das augmentations
            mean_probs = np.mean(all_probs, axis=0)
            
            # Armazenar
            for image_id, prob in zip(image_ids, mean_probs):
                predictions.append({
                    'id': image_id,
                    'target': float(prob)
                })
    
    print(f"✅ {len(predictions)} predições geradas!")
    
    # Salvar CSV
    df = pd.DataFrame(predictions)
    df = df.sort_values('id').reset_index(drop=True)
    df.to_csv(output_file, index=False)
    
    print(f"\n Submissão salva: {output_file}")
    
    # Estatísticas
    print("\n📊 ESTATÍSTICAS:")
    print(f"   Média: {df['target'].mean():.4f}")
    print(f"   Min: {df['target'].min():.4f}")
    print(f"   Max: {df['target'].max():.4f}")
    print(f"   Std: {df['target'].std():.4f}")
    
    print(f"\n📈 DISTRIBUIÇÃO:")
    print(f"   > 0.9: {(df['target'] > 0.9).sum()} ({(df['target'] > 0.9).sum()/len(df)*100:.1f}%)")
    print(f"   0.5-0.9: {((df['target'] >= 0.5) & (df['target'] <= 0.9)).sum()}")
    print(f"   < 0.5: {(df['target'] < 0.5).sum()} ({(df['target'] < 0.5).sum()/len(df)*100:.1f}%)")
    
    return df

In [5]:
if __name__ == '__main__':
    # Configurações
    MODEL_PATH = 'modelo_colab_vision_transformers.pth'
    TEST_DIR = 'dataset/test_images'
    OUTPUT_FILE = 'submission_tta.csv'
    
    # Verificar se arquivos existem
    if not os.path.exists(MODEL_PATH):
        print(f"❌ ERRO: Modelo não encontrado: {MODEL_PATH}")
        print("   Verifique o caminho!")
        exit(1)
    
    if not os.path.exists(TEST_DIR):
        print(f"❌ ERRO: Diretório não encontrado: {TEST_DIR}")
        exit(1)
    
    # Executar TTA
    df = predict_with_tta(MODEL_PATH, TEST_DIR, OUTPUT_FILE)
    
    print("🎉 TTA COMPLETO!")


📂 Carregando modelo: modelo_colab_vision_transformers.pth
✅ Modelo carregado
   Val Acc: 97.20%
   Val ROC-AUC: 0.9969

✅ Dataset: 624 imagens
   TTA: 5 augmentations por imagem

🔮 Gerando predições com TTA...


TTA: 100%|██████████| 78/78 [15:58<00:00, 12.29s/it]

✅ 624 predições geradas!

 Submissão salva: submission_tta.csv

📊 ESTATÍSTICAS:
   Média: 0.6554
   Min: 0.0004
   Max: 0.9999
   Std: 0.4134

📈 DISTRIBUIÇÃO:
   > 0.9: 346 (55.4%)
   0.5-0.9: 60
   < 0.5: 218 (34.9%)
🎉 TTA COMPLETO!
